In [1]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [30]:
# read in data
with open('ssdata/ord.5.tab') as f:
    raw_data = [r.replace('\n', '').split('\t') for r in f.readlines()]

df = pd.DataFrame(raw_data[1:], columns=raw_data[0])

In [26]:
df.head()

,pos,A-2,C-2,D-2,E-2,F-2,G-2,H-2,I-2,K-2,...,V2,W2,Y2,-2,H,E,T,S,B,
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [31]:
# update the last column name
col_names = list(df.columns)
col_names[-1] = 'U'
df.columns = col_names

In [32]:
data = df.copy().drop(['pos', 'H', 'E', 'T', 'S', 'B', 'U'], axis=1)
labels = df.copy()[['H', 'E', 'T', 'S', 'B', 'U']]

In [33]:
print data.shape
print labels.shape

(134814, 105)
(134814, 6)


In [35]:
data.head()

,A-2,C-2,D-2,E-2,F-2,G-2,H-2,I-2,K-2,L-2,...,N2,P2,Q2,R2,S2,T2,V2,W2,Y2,-2
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [34]:
labels.head()

,H,E,T,S,B,U
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


In [3]:
from sklearn.cross_validation import train_test_split
from math import sqrt

In [69]:
xTrain, xTest, yTrain, yTest = train_test_split(np.asarray(data), np.asarray(labels))

In [70]:
print xTrain.shape
print yTrain.shape
print xTest.shape
print yTest.shape

(101110, 105)
(101110, 6)
(33704, 105)
(33704, 6)


### Simple NN Example

In [79]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    (h, w) = shape
    normalizer = 2.0 * sqrt(6) / sqrt(h + w) * 0.1  #factors: correct for uni[0,1], glo, glo, softmax deriv
    return theano.shared(floatX((np.random.random_sample(shape) - 0.5) * normalizer))  #code for using Glorot init

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

def model(X, w_h, w_o):
#     h = T.tanh(T.dot(X, w_h))
    h = T.nnet.sigmoid(T.dot(X, w_h))
    pyx = T.nnet.softmax(T.dot(h, w_o))
    return pyx

X = T.fmatrix()
Y = T.fmatrix()

w_h = init_weights((105, 600))
w_o = init_weights((600, 6))

py_x = model(X, w_h, w_o)
y_pred = T.argmax(py_x, axis=1)

cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
params = [w_h, w_o]

update = RMSprop(cost, params, lr=0.01)

train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

for i in range(101):
    for start, end in zip(range(0, len(xTrain), 128), range(128, len(xTrain), 128)):
        cost = train(xTrain[start:end], yTrain[start:end])
    if i % 10 == 0: 
        print i, np.mean(np.argmax(yTest, axis=1) == predict(xTest))

0 0.580257536197
10 0.611351768336
20 0.626987894612
30 0.632654877759
40 0.638559221457
50 0.645472347496
60 0.6491811061
70 0.65173273202
80 0.655382150487
90 0.657696415856
100 0.660574412533
